# 09 — Gradio Demo: Diabetes Probability (Classification)

In [1]:
# Install latest stable PyCaret 3.x and common GPU-capable libs
# !pip -q install --upgrade pycaret xgboost catboost lightgbm plotly gradio

import sys, sklearn, pycaret, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
print("scikit-learn:", sklearn.__version__)
print("pycaret:", pycaret.__version__)
# 
# Check GPU
try:
    import subprocess, shutil
    if shutil.which("nvidia-smi"):
        print("\nGPU detected:")
        print(subprocess.check_output(["nvidia-smi"], text=True))
    else:
        print("\nNo NVIDIA GPU detected in this runtime. (Colab: Runtime -> Change runtime type -> GPU)")
except Exception as e:
    print("GPU check error:", e)

Python: 3.11.11 (main, Feb  5 2025, 19:11:07) [Clang 19.1.6 ]
Platform: Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.35
scikit-learn: 1.4.2
pycaret: 3.3.2

GPU detected:
Sun Nov  2 20:05:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.65.05              Driver Version: 580.88         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   30C    P8             12W /  200W |    10

In [7]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv")
from pycaret.classification import *
s = setup(df, target='Outcome', session_id=42, normalize=True, use_gpu=True, fold=3)
model = finalize_model(compare_models())
cols = get_config('X_train').columns.tolist()

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1


[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


,Description,Value
0,Session id,42
1,Target,Outcome
2,Target type,Binary
3,Original data shape,"(768, 9)"
4,Transformed data shape,"(768, 9)"
5,Transformed train set shape,"(537, 9)"
6,Transformed test set shape,"(231, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7821,0.8269,0.5878,0.7341,0.6512,0.4959,0.5035,0.0167
ridge,Ridge Classifier,0.7784,0.8260,0.5878,0.7248,0.6473,0.4887,0.4956,0.0167
lda,Linear Discriminant Analysis,0.7747,0.8259,0.5878,0.7156,0.6435,0.4815,0.4877,0.0167
nb,Naive Bayes,0.7523,0.8085,0.5771,0.6762,0.6184,0.4372,0.4435,0.0167
rf,Random Forest Classifier,0.7486,0.7949,0.5716,0.6615,0.6103,0.4268,0.4314,0.2100
qda,Quadratic Discriminant Analysis,0.7486,0.8013,0.5507,0.6825,0.6058,0.4242,0.4323,0.0167
et,Extra Trees Classifier,0.7430,0.7982,0.5344,0.6712,0.5898,0.4069,0.4158,0.1700
gbc,Gradient Boosting Classifier,0.7225,0.7795,0.5715,0.6090,0.5865,0.3787,0.3814,0.0933
lightgbm,Light Gradient Boosting Machine,0.7225,0.7651,0.5610,0.6123,0.5838,0.3765,0.3786,1.6133
knn,K Neighbors Classifier,0.7169,0.7478,0.4867,0.6222,0.5451,0.3441,0.3503,0.0667


In [9]:
import gradio as gr
import pandas as pd
import numpy as np
import socket

def find_free_port(start_port=7860, max_port=7960):
    """Find a free port in the given range"""
    for port in range(start_port, max_port):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(('127.0.0.1', port))
                return port
            except OSError:
                continue
    raise OSError(f"No free ports found in range {start_port}-{max_port}")

def predict_diabetes(**kwargs):
    try:
        # Convert inputs to float and handle missing values
        for key in kwargs:
            kwargs[key] = float(kwargs[key]) if kwargs[key] is not None else 0.0
        
        # Create DataFrame with proper column order
        X = pd.DataFrame([kwargs])[cols]
        
        # Make prediction
        out = predict_model(model, data=X)
        
        # Extract probability and label
        probability = float(out['Score'][0])
        label = int(out['Label'][0])
        
        return probability, label
    except Exception as e:
        print(f"Error in prediction: {str(e)}")
        return 0.0, 0

# Create number inputs with default values from training data mean
inputs = []
train_means = get_config('X_train').mean()
for col in cols:
    inputs.append(
        gr.Number(
            label=col,
            value=float(train_means[col]),  # Set default value
            precision=2
        )
    )

# Create the interface
demo = gr.Interface(
    fn=lambda *vals: predict_diabetes(**{c:v for c,v in zip(cols, vals)}),
    inputs=inputs,
    outputs=[
        gr.Number(label="Probability of Diabetes", precision=3),
        gr.Number(label="Predicted Class (0: No, 1: Yes)")
    ],
    title="Diabetes Classifier (PyCaret)",
    description="Enter patient measurements to predict diabetes probability."
)

# Find an available port
try:
    port = find_free_port()
    print(f"Starting server on port {port}")
    
    # Launch with the available port
    demo.launch(
        server_name="127.0.0.1",
        server_port=port,
        share=False,
        show_error=True
    )
except Exception as e:
    print(f"Error launching server: {str(e)}")

Starting server on port 7861
* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Error in prediction: 'Score'


Error in prediction: 'Score'


Error in prediction: 'Score'


Error in prediction: 'Score'
